In [1]:
%load_ext autoreload
%autoreload 2

Current issue 3/4/25: Jubilee pressure cell doesn't have pump set up right, throwing errors on the AFL side of things. Integration otherwise seems to work OK

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
from science_jubilee.tools import PneumaticSampleLoader
import time
import numpy as np

import logging
import sys


In [3]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
gcode cmd:  M409 K"move.axes[].homed"
response in connect:  [True, True, True, True]
gcode cmd:  M409 K"move.axes[]"
gcode cmd:  T
gcode cmd:  M409 K"tools"
gcode cmd:  M409 K"tools"
gcode cmd:  M409 K"move.axes"
gcode cmd:  G90
gcode cmd:  G90


In [4]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [8]:
safe_pos = (5, 200, 100)
cell_pos = (200, 50, 20)
url = 'http://192.168.1.4'
port = '5000'
name = 'PSL'
username = 'test'
password = 'domo_arigato'

In [9]:
jubilee.move_to(x = safe_pos[0], y = safe_pos[1], z = safe_pos[2])

gcode cmd:  G90
gcode cmd:  G0 Z100.00 X5.00 Y200.00   F6000.00 
gcode cmd:  M400


In [10]:
psl = PneumaticSampleLoader.PneumaticSampleLoader(url, port, name, cell_pos, safe_pos, username, password)

status r code 200
status:  b'["State: INITIALIZED","Arm State: UNKNOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'


In [11]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")

In [12]:
jubilee.load_tool(syringe_10)
mix_syringe = syringe_10

gcode cmd:  M409 K"tools"


In [13]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)
mix_syringe.set_pulsewidth(mix_syringe.full_position+250, s = 500)
mix_syringe.load_syringe(6200, mix_syringe.full_position+250)

Loaded syringe, remaining volume 0 uL
Loaded syringe, remaining volume 6200 uL


In [14]:
samples = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples.manual_offset([(31.1, 169.0), (121.1, 169.0), (121.1, 113.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [15]:
psl.arm_state

'UNKNOWN'

In [16]:
psl.get_cell_state()

status r code 200
status:  b'["State: INITIALIZED","Arm State: UNKNOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'


'INITIALIZED'

In [17]:
jubilee.load_tool(psl)

In [25]:
psl.rinse_cell()

status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': True}","Pushing with syringe...","[250306-15:17:23] Load stopped at voltage mean = 3.2751725196838377 and stdev = 1.0367813813585773"]\n'
status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': True}","Pushing with syringe...","[250306-15:17:23] Load stopped at voltage mean = 3.2751725196838377 and stdev = 1.0367813813585773"]\n'
status r code 200
status:  b'["State: RINSING","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False,

In [24]:
psl.unpause_queue()

In [24]:
psl.load_sample(mix_syringe, samples['A1'], 500)

status r code 200
status:  b'["State: RINSED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","No action yet..."]\n'
gcode cmd:  G90
gcode cmd:  G0 Z100.00 X5.00 Y200.00   F6000.00 
gcode cmd:  M400
status r code 200
status:  b'["State: RINSED","Arm State: DOWN","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': False, \'arm-down\': True, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': False, \'postsample\': False}","Not Rinsing","No action yet..."]\n'
status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsampl

In [35]:
jubilee.tools.values()

dict_values([{'name': '10cc_1', 'tool': <science_jubilee.tools.HTTPSyringe.HTTPSyringe object at 0x7f07880784d0>}, {'name': 'PSL', 'tool': <science_jubilee.tools.PneumaticSampleLoader.PneumaticSampleLoader object at 0x7f07880f3810>}])

In [44]:
jubilee.active_tool_index

response text in gcode:  No tool is selected



-1

In [48]:
jubilee.gcode('M114')

response text in gcode:  X:5.000 Y:200.000 Z:100.000 U:0.000 E:0.000 Count 41000 -39000 160000 0 Machine 5.000 200.000 100.000 0.000 Bed comp 0.000



'X:5.000 Y:200.000 Z:100.000 U:0.000 E:0.000 Count 41000 -39000 160000 0 Machine 5.000 200.000 100.000 0.000 Bed comp 0.000\n'

In [50]:
jubilee.gcode('T')

response text in gcode:  No tool is selected



'No tool is selected\n'

In [ ]:
jubilee._get_tool_index(jubilee.active_tool_index

In [42]:
jubilee.park_tool()

response text in gcode:  X:10.000 Y:200.000 Z:200.000 U:0.000 E:0.000 Count 42000 -38000 320000 0 Machine 10.000 200.000 200.000 0.000 Bed comp 0.000

response text in gcode:  

response text in gcode:  No tool is selected



In [41]:
jubilee.active_tool_index

response text in gcode:  No tool is selected



-1

In [20]:
print(jubilee)

In [26]:
jubilee.session